In [4]:
import numpy as np
import pandas as pd
import sys
import numpy
from collections import defaultdict
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.decomposition import NMF

In [2]:
header = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=header)
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items))

Number of users = 943 | Number of movies = 1682


In [3]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df, test_size=0.25)
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = 1

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = 1

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
def nmf_k(k):
#get SVD components from train matrix. Choose k.
    model = NMF(n_components=k, init='random', random_state=0)
    w = model.fit_transform(train_data_matrix)
    h=model.components_
    return w,h

In [21]:
    def auc(w,h, test_data, train_data):
        """
          Computes the Area Under Curve (AUC) on `test_data`.
          `test_data` is an array of (user_index, item_index) tuples.
          During this computation we ignore users and items
          that didn't appear in the training data, to allow
          for non-overlapping training and testing sets.
        """
        test_dict, test_users, test_items = _data_to_dict(test_data)
        _train_dict, _train_users, _train_items = _data_to_dict(train_data)
        auc_values = []
        z = 0
        for user in test_dict.keys():
            if user in _train_users:
                auc_for_user = 0.0
                n = 0
                predictions = predict_for_user(w,h,user-1)
                for pos_item in test_dict[user]:
                    if pos_item in _train_items:
                        for neg_item in _train_items:
                            if neg_item not in test_dict[user] and neg_item not in _train_dict[user]:
                                n += 1
                                #print(user)
                                if predictions[pos_item-1] > predictions[neg_item-1]:
                                    auc_for_user += 1
                if n > 0:
                    auc_for_user /= n
                    auc_values.append(auc_for_user)
                z += 1
                if z % 100 == 0 and len(auc_values) > 0:
                    sys.stderr.write("\rCurrent AUC mean (%s samples): %0.5f" % (str(z), numpy.mean(auc_values)))
                    sys.stderr.flush()
        sys.stderr.write("\n")
        sys.stderr.flush()
        return numpy.mean(auc_values)

In [22]:
def predict_for_user(w,h, n):
    return np.dot(w[n], h)
    #return np.dot(np.dot(u[n], s_diag_matrix), vt)

In [24]:
    def _data_to_dict(data):
        data_dict = defaultdict(list)
        items = set()
        for user in data.iterrows():
            data_dict[int(user[1][0])].append(int(user[1][1]))
            items.add(int(user[1][1]))
        return data_dict, set(data_dict.keys()), items

In [25]:
aucs=[]
w5,h5=nmf_k(5)
aucs.append(auc(w5,h5,test_data, train_data))

Current AUC mean (900 samples): 0.91745


In [26]:
w10,h10=nmf_k(10)
aucs.append(auc(w10,h10,test_data, train_data))

Current AUC mean (900 samples): 0.92319


In [27]:
w20,h20=nmf_k(20)
aucs.append(auc(w20,h20,test_data, train_data))

Current AUC mean (900 samples): 0.92638


In [28]:
w30,h30=nmf_k(30)
aucs.append(auc(w30,h30,test_data, train_data))

Current AUC mean (900 samples): 0.91775


In [29]:
aucs

[0.91787694144686927,
 0.92361092400181044,
 0.92691969295417398,
 0.91842521899305052]

In [30]:
w50,h50=nmf_k(50)
aucs.append(auc(w50,h50,test_data, train_data))

Current AUC mean (900 samples): 0.90118


In [31]:
w100,h100=nmf_k(100)
aucs.append(auc(w100,h100,test_data, train_data))

Current AUC mean (900 samples): 0.86904


In [32]:
aucs

[0.91787694144686927,
 0.92361092400181044,
 0.92691969295417398,
 0.91842521899305052,
 0.90178557741196819,
 0.87008136205324826]